In [2]:
# Step 1 : Import the necessary packages for implementing recurrent neural network
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable
import numpy as np
import pylab as pl

In [3]:
# Step 2 : Set the model hyper parameters with the size of input layer set to 7.
# There will be 6 context neurons and 1 input neuron for creating target sequence
dtype = torch.FloatTensor
input_size, hidden_size, output_size = 7, 6, 1
epochs = 300
seq_length = 20
lr = 0.1
data_time_steps = np.linspace(2, 10, seq_length + 1)
data = np.sin(data_time_steps)
data.resize((seq_length + 1, 1))

x = Variable(torch.Tensor(data[:-1]).type(dtype), requires_grad=False)
y = Variable(torch.Tensor(data[1:]).type(dtype), requires_grad=False)

In [4]:
# Step 3 : Weights are initialized in the recurrent neural network using normal distribution with zero mean.
# W1 will represent acceptance of input variables and W2 will represent the output which is generated.
w1 = torch.Tensor(input_size, hidden_size).type(dtype)
init.normal(w1, 0.0, 0.4)
w1 = Variable(w1, requires_grad=True)

w2 = torch.Tensor(hidden_size, output_size).type(dtype)
init.normal(w2, 0.3, 0.0)
w2 = Variable(w2, requires_grad=True)

C:\Users\ui\AppData\Local\Temp\ipykernel_16992\816415515.py:4: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(w1, 0.0, 0.4)
C:\Users\ui\AppData\Local\Temp\ipykernel_16992\816415515.py:8: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(w2, 0.3, 0.0)


In [5]:
# Step 4 : Create a function for feed forward which uniquely defines the neural network
def forward(input, context_state, w1, w2):
    xh = torch.cat((input, context_state), 1)
    context_state = torch.tanh(xh.mm(w1))
    out = context_state.mm(w2)
    return (out, context_state)

In [6]:
# Step 5: Start training procedure of recurrent neural network's sine wave implementation.
# The outer loop iterates over each loop and the inner loop iterates through the element of sequence.
# here, we will also compute Mean Square Error which helps in the prediction of continuous variables
for i in range(epochs):
    total_loss = 0
    context_state = Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=True)
    for j in range(x.size(0)):
        input = x[j:(j + 1)]
        target = y[j:(j + 1)]
        (pred, context_state) = forward(input, context_state, w1, w2)
        loss = (pred - target).pow(2).sum() / 2
        total_loss += loss
        loss.backward()
        w1.data -= lr * w1.grad.data
        w2.data -= lr * w2.grad.data
        w1.grad.data.zero_()
        w2.grad.data.zero_()
        context_state = Variable(context_state.data)
    if i % 10 == 0:
        print('Epoch: {0} loss {1}'.format(i, total_loss.data))

context_state = Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=False)
predictions = []

for i in range(x.size(0)):
    input = x[i: i + 1]
    (pred, context_state) = forward(input, context_state, w1, w2)
    context_state = context_state
    predictions.append(pred.data.numpy().ravel()[0])

Epoch: 0 loss 4.71427059173584
Epoch: 10 loss 0.20524361729621887
Epoch: 20 loss 0.1951163411140442
Epoch: 30 loss 0.18877293169498444
Epoch: 40 loss 0.1835704743862152
Epoch: 50 loss 0.1791139841079712
Epoch: 60 loss 0.1754252165555954
Epoch: 70 loss 0.17254222929477692
Epoch: 80 loss 0.17045822739601135
Epoch: 90 loss 0.16913776099681854
Epoch: 100 loss 0.16853761672973633
Epoch: 110 loss 0.16862095892429352
Epoch: 120 loss 0.16936852037906647
Epoch: 130 loss 0.17079435288906097
Epoch: 140 loss 0.17298129200935364
Epoch: 150 loss 0.17617522180080414
Epoch: 160 loss 0.18099556863307953
Epoch: 170 loss 0.18862691521644592
Epoch: 180 loss 0.19962133467197418
Epoch: 190 loss 0.21015363931655884
Epoch: 200 loss 0.21716775000095367
Epoch: 210 loss 0.22585006058216095
Epoch: 220 loss 0.238516703248024
Epoch: 230 loss 0.2508842349052429
Epoch: 240 loss 0.2590409219264984
Epoch: 250 loss 0.26224520802497864
Epoch: 260 loss 0.26093655824661255
Epoch: 270 loss 0.2558135688304901
Epoch: 280 loss

In [7]:
# Step 6 : Plot the sine wave as the way it is needed
pl.scatter(data_time_steps[:-1], x.data.numpy(), s = 90, label = 'Actual')
pl.scatter(data_time_steps[1:], predictions)
pl.legend()
pl.show()

: 

: 